### **Load Dataset**

In [1]:
# Install the HuggingFace Datasets and Evaluate libraries

! pip install --quiet datasets evaluate accelerate

In [2]:
# Load the Amharic news text classification dataset
from datasets import load_dataset

news_dataset = load_dataset("rasyosef/amharic-news-category-classification")
news_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label'],
        num_rows: 49971
    })
})

In [3]:
raw_datasets = news_dataset['train'].train_test_split(train_size=0.8, seed=42)

print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label'],
        num_rows: 39976
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label'],
        num_rows: 9995
    })
})


In [4]:
print(raw_datasets['train'][0])
print(raw_datasets['test'][0])
print(raw_datasets['train'].features)

{'headline': 'ዘላቂ  ልማትን ለማረጋገጥ  ያለመው  የአዲስ  አበባ የልማት አጀንዳ  ተቀባይነት አገኘ', 'category': 'ፖለቲካ', 'date': 'July 17, 2015', 'views': 'Unknown', 'article': 'በዛሬው ዕለት 3ኛው የፋይናንስ ጉባኤ ለልማት ጉባኤ ማጠቃለያ ላይ የኤፌዴሪ ጠቅላይ ሚኒስትርና የጉባኤው ፕሬዚዳንት ኃይለማሪያም ደሳለኝ እንደገለጹት ዘላቂ ልማት ለማረጋገጥ የሚያስችለውን የአዲስ አበባ የልማት አጀንዳ መሪ ዕቅድ ተግባራዊ ከአራት ቀን ጉባኤ በኋላ ስምምነትን አግኝቷል ።እንደ ጠቅላይ ሚኒስትር ኃይለማሪያም ገለጻ የፋይናንስ ለልማተ ጉባኤ የተካሄደበት \xa0ሳምንት በዓለም ልማት ታሪክ ላይ ጉልህ ሥፍራ እንዳለው በመጠቆም \xa0በንም \xa0በልማት ወደ ኋላ አይቀርም የሚለውን መርህ \xa0በታላቅ ጉጉት እየጠበቀው ይገኛል ብለዋል ።በጉባኤ \xa0ያደጉ አገራት አብዛኛውን የልማት ድጋፍ ገና በማደግ ላይ ላሉ አገራት ለመመደብ \xa0መስማማታቸው \xa0የጉባኤ ትልቅ ውጤት ነው ያሉት ጠቅላይ ሚኒስትሩ \xa0ያደጉ አገራት ከአጠቃላይ አገራዊ ገቢያቸው \xa0ከዜሮነጥብ7 የሚሆነውን \xa0ለልማት ድጋፍ እንዲያውሉና ከዚህም ውስጥ \xa0ከ ዜሮ ነጥብ 15 እስከ ዜሮ ነጥብ ሃያ ድረስ \xa0በድህነት ጫፍ ላይ ለሚገኙ አገራት እንደሚመደብ አስረድተዋል ።ጉባኤው \xa0የግሉ ዘርፍ \xa0ለልማት ያለውን አስተዋጽኦ አጉልቶ ማውጣቱን የጠቆሙት ጠቅላይ ሚኒስትሩ \xa0 የመንግሥትና የግል ዘርፉ የኢንቨስትመንት መዋለ ንዋይ ትክክለኛ የኢኮኖሚ ዕድገትን እንደሚያመጣ \xa0አብራርተዋል ።ኢትዮጵያ \xa0እኤአ በ2025 \xa0መካከለኛ \xa0ገቢ ካላቸው አገራት ለማሰለፍ ከወዲሁ ውጥን ተይዞ እየተሠራ መሆኑንየጠቀሱት ጠቅላይ ሚኒሰትሩ የውጭ ቀ

In [5]:
# Remove articles that are too short

raw_datasets = raw_datasets.filter(lambda x: x['word_len'] >= 32)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label'],
        num_rows: 38966
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label'],
        num_rows: 9735
    })
})

In [6]:
categories = raw_datasets['train'].features['label'].names
categories

['ሀገር አቀፍ ዜና', 'መዝናኛ', 'ስፖርት', 'ቢዝነስ', 'ዓለም አቀፍ ዜና', 'ፖለቲካ']

In [7]:
# Concatenate the title and article
raw_datasets = raw_datasets.map(lambda x: {"full_article" : x["headline"] + "\n" + x["article"]})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label', 'full_article'],
        num_rows: 38966
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label', 'full_article'],
        num_rows: 9735
    })
})

### **Preprocessing the dataset**

In [8]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenize the dataset

def tokenize_function(example):
  return tokenizer(example['full_article'], truncation=True, max_length=512)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

# Use a data collator to apply dynamic padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

print(tokenized_datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/9735 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label', 'full_article', 'input_ids', 'attention_mask'],
        num_rows: 38966
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label', 'full_article', 'input_ids', 'attention_mask'],
        num_rows: 9735
    })
})


### **Finetuning the Model**

In [9]:
# Load the model

from transformers import AutoModelForSequenceClassification

# roberta-base

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=len(categories),
    id2label = {i: lbl for i, lbl in enumerate(categories)},
    label2id = {lbl: i for i, lbl in enumerate(categories)},
    device_map="cuda"
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

batch_size = 16
gradient_accumulation_steps = 4
epochs = 5

training_args = TrainingArguments(
    output_dir=checkpoint+"-finetuned",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    seed=42,
)

In [11]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  precision = metric2.compute(predictions=predictions, references=labels, average='macro')["precision"]
  recall = metric3.compute(predictions=predictions, references=labels, average='macro')["recall"]
  f1 = metric4.compute(predictions=predictions, references=labels, average='macro')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }

compute_metrics(([[1,0], [0,1]], [0,1]))

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.562200,0.365156,0.862250,0.782529,0.865374,0.812864


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.562200,0.365156,0.862250,0.782529,0.865374,0.812864
2,0.324000,0.321566,0.880637,0.859052,0.836564,0.844553
3,0.250300,0.325716,0.883102,0.882634,0.793801,0.817918
4,0.190200,0.321911,0.894196,0.858177,0.876819,0.866780
5,0.141000,0.323280,0.898408,0.876105,0.875467,0.875634


TrainOutput(global_step=3045, training_loss=0.293547734603506, metrics={'train_runtime': 2727.4718, 'train_samples_per_second': 71.432, 'train_steps_per_second': 1.116, 'total_flos': 5.12635130584295e+16, 'train_loss': 0.293547734603506, 'epoch': 5.0})

### **Model Predictions**

In [13]:
# Load metrics and evaluate the model

from torch.utils.data import DataLoader

eval_dataset = tokenized_datasets["test"].remove_columns([
    'headline', 'category', 'date', 'views',
    'article', 'link', 'word_len', 'full_article'
    ]).rename_column("label", "labels").with_format("torch")

print(eval_dataset.column_names)

eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
)

import evaluate
import torch

y_pred, y_test = [], []

metric = evaluate.load("f1")
model.eval()
for batch in eval_dataloader:
  batch = {k: v.to('cuda') for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch["labels"])
  y_pred.extend(predictions.cpu().numpy())
  y_test.extend(batch["labels"].cpu().numpy())
metric.compute(average='macro')

['labels', 'input_ids', 'attention_mask']


{'f1': 0.8756344650022968}

In [ ]:
len(y_pred), len(y_test)

In [15]:
metric.compute(predictions=y_pred, references=y_test, average='weighted')

{'f1': 0.8986341914792896}

In [16]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred)

array([[3516,    6,   12,  135,   80,  209],
       [  15,   90,    1,    0,    2,    1],
       [   3,    0, 1946,    0,    1,    2],
       [ 121,    1,    0,  584,    0,   72],
       [  67,    1,    1,    2, 1024,   12],
       [ 150,    6,    3,   80,    6, 1586]])

In [17]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      3958
           1       0.87      0.83      0.85       109
           2       0.99      1.00      0.99      1952
           3       0.73      0.75      0.74       778
           4       0.92      0.93      0.92      1107
           5       0.84      0.87      0.85      1831

    accuracy                           0.90      9735
   macro avg       0.88      0.88      0.88      9735
weighted avg       0.90      0.90      0.90      9735

